In [1]:
# Disable the IPython pager
# https://gist.github.com/minrk/7715212
from IPython.core import page
page.page = print

# 06 - Property Based Testing (Bonus)

SW testing joke
> A QA engineer walks into a bar. Orders a beer. Orders 0 beers. Orders 99999999999 beers. Orders a lizard. Orders -1 beers. Orders a ueicbksjdhd. 

> First real customer walks in and asks where the bathroom is. The bar bursts into flames, killing everyone.

From [hypothesis documentation](https://hypothesis.readthedocs.io/en/latest):
> [Hypothesis](https://hypothesis.works) is a Python library for creating unit tests which are simpler to write and more powerful when run, finding edge cases in your code you wouldn’t have thought to look for. It is stable, powerful and easy to add to any existing test suite.

In [2]:
import pytest
import hypothesis

The Hypothesis library provides 
* strategies for generating input data based on given criteria
* tools for integration those into property based tests

Sligthly modified [definition](https://hypothesis.readthedocs.io/en/latest/):

| Example based tests | Property based tests |
| :------------------ | :------------------- |
| 1. Set up *some example* data | 1. for **all data** matching some specification |
| 2. Perform some operations on the data | 2. Perform some operations on the data |
| 3. Assert something *specific* about the result | 3. Assert something **generic** about the result |
    

## Test data generators

In [3]:
from hypothesis.strategies import floats, integers, text

In [4]:
f = floats()

In [5]:
[f.example() for _ in range(10)]

[1.175494351e-38,
 -inf,
 inf,
 nan,
 nan,
 -0.5,
 inf,
 -1.132695145266593e+290,
 -1e-05,
 5.309959770981525e+276]

In [6]:
print(text().example())

򠈇.


See more examples under [What you can generate and how](https://hypothesis.readthedocs.io/en/latest/data.html)

## Example use-case
Let's assume we need to write a function that returns the 2nd largest value in a list of integers

In [7]:
import math

In [8]:
def second_largest(values):
    return values[-2]

Is this a good implementation?

To find out, we can try a couple of example tests

In [9]:
second_largest([1, 2, 3, 4, 5])

4

In [10]:
second_largest([1, 2, 3, 4])

3

Seems ok? Except it isn't.

These tests are too "weak". Let's use Hypothesis

In [11]:
%pycat test_stats.py

import pytest
from hypothesis import given
from hypothesis.strategies import lists, integers
from .stats import second_largest


def test_example():
    """Example based tests.

    With poorly chosen examples"""
    assert 4 == second_largest([1, 2, 3, 4, 5])
    assert 3 == second_largest([1, 2, 3, 4])


@given(values=lists(integers()))
def test_hypothesis(values):
    """Property based tests."""
    distinct_values = set(values)
    if len(distinct_values) > 1:
        # valid usage -> test expected properties
        v = second_largest(values)
        # returned value is present in the list
        assert v in values  
        # there is exactly one value greater than v
        assert 1 == len([x for x in distinct_values if x > v]) 
    else:
        # invalid usage -> test expected outcome
        with pytest.raises(ValueError):
            second_largest(values)



In [12]:
!pytest test_stats.py

============================= test session starts =============================
platform win32 -- Python 3.6.9, pytest-5.1.2, py-1.8.0, pluggy-0.13.0
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase('C:\\Users\\Petr\\Documents\\GitHub\\pydata_nyc_2019\\solutions\\.hypothesis\\examples')
rootdir: C:\Users\Petr\Documents\GitHub\pydata_nyc_2019\solutions
plugins: hypothesis-4.36.2
collected 2 items

test_stats.py ..                                                         [100%]

============================== 2 passed in 0.28s ==============================


## Pandas and Numpy types
Read more on [Hypothesis for the Scientific Stack](https://hypothesis.readthedocs.io/en/latest/numpy.html)

## See also
* [Quick Start Guide](https://hypothesis.readthedocs.io/en/latest/quickstart.html)
* [Introductory articles](https://hypothesis.works/articles/)